In [50]:
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torch.optim as optim

In [5]:
train_df = pd.read_csv("./Data/train_relationships.csv")
test_df = pd.read_csv("./Data/sample_submission.csv")
# 将测试集样本成对放在一起
test_df_list = []      # 测试集图片文件名
for i in range(0, len(test_df)):
    test_df_list.append((test_df.iloc[i]['img_pair']).split('-'))

用一张测试集中的图片跑通神经网络，不用训练的神经网络，学习人脸的一个特征向量，然后基于该特征向量计算出人脸之间的相似度

pytorch实现简单的cnn

In [49]:
# Define the network,固定了输出和输出的
batch_size = 128
learning_rate = 1e-2
num_epoches = 20  

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input channel, 6 output channel, 5*5 square convolution
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        
        self.fc1 = nn.Linear(16*5*5,120)  # 弄明白上一层的卷积输出是什么
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        
        x = x.view(-1,self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

cnn_net = Net()


# 判断是否使用GPU
use_gpu = torch.cuda.is_available()  # 判断是否有GPU加速
if use_gpu:
    model = cnn_net.cuda()

#  Define the loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=learning_rate)

In [ ]:
# 开始训练：
for epoch in  range(num_epoches):
    print('epoch{}'.format(epoch+1))
    print('*'*10)
    running_loss = 0.0
    running_acc = 0.0
    
    for i,data in data:
        img = 
        label = 
        if use_gpu:
            img = img_cuda()
            label = label.cuda()
        img = Variable(img)
        label = Variable(label)
        out = model(img)
        loss = criterion(out,label)
        
        running_loss +=loss.data[0]*label.size(0) #这句话是为什么
        _, pred = torch.max(out,1)
        num_correct = (pred == label).sum()
        accuracy = (pred == label).float().mean()# 为什么是这样求
        running_acc += num_correct.data[0]
         
        # 向后传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 300 == 0:
            print('[{}/{}] Loss:{:.6f}, Acc: {:.f}'.format(epoch+1, num_epoches, 
                                                          running_loss/(batch_size*i),
                                                          running_acc/(batch_size*i)))
        model.eval()
        eval_loss = 0
        eval_acc = 0
        
        for data in test :
            img =
            label =
            if use_gpu:
                img = Variable(img, volatile=True).cuda()
                label = Variable(label, volatitle=True).cuda()
            else:
                img = Variable(img, volatile=True)
                label = Variable(label, volatitle=True)
            out = model(img)
            loss = criterion(out, label)
            eval_loss +=loss.data[0]*label.size(0)
            _,pred = torch.max(out,1)
            num_correct = (pred==label).sum()
            eval_acc + = num_correct.data[0]
        print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(
        test_dataset)), eval_acc / (len(test_dataset))))

# 保存模型
torch.save(model.state_dict(),'./cnn.pth')

In [40]:
# Updata the weight
optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()
output = net(input)
loss = criterion(output,target)
loss.backward()
optimizer.step()

In [39]:
#反向传播的三个步骤
optimizer.zero_grad()
loss.backward()
optimizer.step()

In [ ]:
# 训练cnn网络

In [45]:
logger = Logger('./logs')

NameError: name 'Logger' is not defined

# 使用其他网络结构

In [25]:
model_name = "resnet"

# Number of classes in the dataset
num_classes = 2

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 15

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

In [26]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()
    val_acc_history = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [27]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [28]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, 2, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [29]:
import torch  
from torch.autograd import Variable  
import torch.nn as nn  
from graphviz import Digraph
os.environ["PATH"] += os.pathsep + 'D:/Anaconda3/Library/bin/graphviz-2.38/bin'
def make_dot(var, params=None):
    if params is not None:
        assert isinstance(params.values()[0], Variable)
        param_map = {id(v): k for k, v in params.items()}
 
    node_attr = dict(style='filled',
                     shape='box',
                     align='left',
                     fontsize='12',
                     ranksep='0.1',
                     height='0.2')
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()
 
    def size_to_str(size):
        return '('+(', ').join(['%d' % v for v in size])+')'
 
    def add_nodes(var):
        if var not in seen:
            if torch.is_tensor(var):
                dot.node(str(id(var)), size_to_str(var.size()), fillcolor='orange')
            elif hasattr(var, 'variable'):
                u = var.variable
                name = param_map[id(u)] if params is not None else ''
                node_name = '%s\n %s' % (name, size_to_str(u.size()))
                dot.node(str(id(var)), node_name, fillcolor='lightblue')
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, 'next_functions'):
                for u in var.next_functions:
                    if u[0] is not None:
                        dot.edge(str(id(u[0])), str(id(var)))
                        add_nodes(u[0])
            if hasattr(var, 'saved_tensors'):
                for t in var.saved_tensors:
                    dot.edge(str(id(t)), str(id(var)))
                    add_nodes(t)
    add_nodes(var.grad_fn)
    return dot
if __name__ == '__main__':  
    x = Variable(torch.randn(1, 3, 224,224))  
    y = model_ft(x)  
    g = make_dot(y)  
    g.view()  
  
    params = list(model_ft.parameters())  
    k = 0  
    for i in params:  
        l = 1  
        print("该层的结构：" + str(list(i.size())))  
        for j in i.size():  
            l *= j  
        print("该层参数和：" + str(l))  
        k = k + l  
    print("总参数数量和：" + str(k))

该层的结构：[64, 3, 7, 7]
该层参数和：9408
该层的结构：[64]
该层参数和：64
该层的结构：[64]
该层参数和：64
该层的结构：[64, 64, 3, 3]
该层参数和：36864
该层的结构：[64]
该层参数和：64
该层的结构：[64]
该层参数和：64
该层的结构：[64, 64, 3, 3]
该层参数和：36864
该层的结构：[64]
该层参数和：64
该层的结构：[64]
该层参数和：64
该层的结构：[64, 64, 3, 3]
该层参数和：36864
该层的结构：[64]
该层参数和：64
该层的结构：[64]
该层参数和：64
该层的结构：[64, 64, 3, 3]
该层参数和：36864
该层的结构：[64]
该层参数和：64
该层的结构：[64]
该层参数和：64
该层的结构：[128, 64, 3, 3]
该层参数和：73728
该层的结构：[128]
该层参数和：128
该层的结构：[128]
该层参数和：128
该层的结构：[128, 128, 3, 3]
该层参数和：147456
该层的结构：[128]
该层参数和：128
该层的结构：[128]
该层参数和：128
该层的结构：[128, 64, 1, 1]
该层参数和：8192
该层的结构：[128]
该层参数和：128
该层的结构：[128]
该层参数和：128
该层的结构：[128, 128, 3, 3]
该层参数和：147456
该层的结构：[128]
该层参数和：128
该层的结构：[128]
该层参数和：128
该层的结构：[128, 128, 3, 3]
该层参数和：147456
该层的结构：[128]
该层参数和：128
该层的结构：[128]
该层参数和：128
该层的结构：[256, 128, 3, 3]
该层参数和：294912
该层的结构：[256]
该层参数和：256
该层的结构：[256]
该层参数和：256
该层的结构：[256, 256, 3, 3]
该层参数和：589824
该层的结构：[256]
该层参数和：256
该层的结构：[256]
该层参数和：256
该层的结构：[256, 128, 1, 1]
该层参数和：32768
该层的结构：[256]
该层参数和：256
该层的结构：[256]
该层参数和：256
该层的

In [30]:
# Send the model to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 fc.weight
	 fc.bias


# Loading Data

In [49]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


NameError: name 'data_dir' is not defined

#  Construct Face Data Set

In [51]:
class FaceDataSet(Dataset):

SyntaxError: unexpected EOF while parsing (<ipython-input-51-7d94705dc653>, line 1)

In [17]:
from PIL import Image, ImageDraw
import face_recognition

# Load the jpg file into a numpy array
image = face_recognition.load_image_file(test_images_path[0])

# Find all facial features in all the faces in the image
face_landmarks_list = face_recognition.face_landmarks(image)

print("I found {} face(s) in this photograph.".format(len(face_landmarks_list)))

# Create a PIL imagedraw object so we can draw on the picture
pil_image = Image.fromarray(image)
d = ImageDraw.Draw(pil_image)

for face_landmarks in face_landmarks_list:

    # Print the location of each facial feature in this image
    for facial_feature in face_landmarks.keys():
        print("The {} in this face has the following points: {}".format(facial_feature, face_landmarks[facial_feature]))

    # Let's trace out each facial feature in the image with a line!
    for facial_feature in face_landmarks.keys():
        d.line(face_landmarks[facial_feature], width=5)

# Show the picture
pil_image.show()

I found 1 face(s) in this photograph.
The chin in this face has the following points: [(-11, 60), (-12, 90), (-11, 121), (-7, 152), (2, 180), (20, 202), (43, 219), (72, 227), (100, 227), (121, 222), (134, 206), (140, 184), (145, 161), (151, 140), (157, 119), (158, 101), (157, 83)]
The left_eyebrow in this face has the following points: [(25, 43), (43, 31), (65, 27), (88, 31), (108, 42)]
The right_eyebrow in this face has the following points: [(132, 46), (143, 42), (154, 41), (163, 44), (166, 52)]
The nose_bridge in this face has the following points: [(120, 61), (124, 77), (128, 92), (132, 109)]
The nose_tip in this face has the following points: [(95, 121), (108, 127), (120, 131), (130, 129), (138, 124)]
The left_eye in this face has the following points: [(44, 59), (58, 55), (72, 55), (83, 63), (71, 65), (57, 64)]
The right_eye in this face has the following points: [(128, 68), (139, 64), (149, 66), (154, 73), (148, 75), (138, 73)]
The top_lip in this face has the following points: 

In [54]:
time = '11:22'

In [65]:
int("00")

0

In [64]:
str(22)

'22'

使用sklearn中的Perception model

In [42]:
from sklearn.linear_model import Perceptron
import numpy as np

x1 = np.array([[3],[3]])
x2 = np.array([[4],[3]])
x3 = np.array([[1],[1]])
x = np.hstack([x1,x2,x3]).T
y = np.array([[1],[1],[-1]])

# sklearn中的感知机分类器
clf = Perceptron(tol=1e-3, random_state=0)
clf.fit(x,y) #训练
clf.score(x,y)  #训练集结果

d:\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1.0